In [2]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import time

In [3]:
# read in all the words
words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [4]:
len(words)

32033

In [5]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [6]:
# build the dataset

block_size = 3 # context length: how many characters do we take to predict the next one?
X, Y = [], []

for w in words:  
  #print(w)
  context = [0] * block_size
  for ch in w + '.':
    ix = stoi[ch]
    X.append(context)
    Y.append(ix)
    #print(''.join(itos[i] for i in context), '--->', itos[ix])
    context = context[1:] + [ix] # crop and append
  
X = torch.tensor(X)
Y = torch.tensor(Y)

In [7]:
# build the dataset
block_size = 5 # context length: how many characters do we take to predict the next one?

def build_dataset(words):  
  X, Y = [], []
  for w in words:

    #print(w)
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      #print(''.join(itos[i] for i in context), '--->', itos[ix])
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])

torch.Size([182625, 5]) torch.Size([182625])
torch.Size([22655, 5]) torch.Size([22655])
torch.Size([22866, 5]) torch.Size([22866])


In [8]:
Xtr

tensor([[ 0,  0,  0,  0,  0],
        [ 0,  0,  0,  0, 25],
        [ 0,  0,  0, 25, 21],
        ...,
        [ 0,  8, 15, 12,  4],
        [ 8, 15, 12,  4,  1],
        [15, 12,  4,  1, 14]])

In [9]:
Xtr.shape, Xtr

(torch.Size([182625, 5]),
 tensor([[ 0,  0,  0,  0,  0],
         [ 0,  0,  0,  0, 25],
         [ 0,  0,  0, 25, 21],
         ...,
         [ 0,  8, 15, 12,  4],
         [ 8, 15, 12,  4,  1],
         [15, 12,  4,  1, 14]]))

In [10]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([228146, 3]), torch.int64, torch.Size([228146]), torch.int64)

In [11]:
C = torch.randn((27, 2))

In [12]:
emb = C[X]
emb.shape

torch.Size([228146, 3, 2])

In [13]:
W1 = torch.randn((6, 100))
b1 = torch.randn(100)

In [14]:
h = torch.tanh(emb.view(emb.shape[0], 6) @ W1 + b1)

In [15]:
h

tensor([[ 0.8265, -0.6199,  0.0622,  ..., -0.6910, -0.7878, -0.0085],
        [ 0.9904, -0.9756, -0.4458,  ..., -0.9560,  0.3492, -0.9996],
        [ 0.9978, -0.9987,  0.4483,  ...,  0.9224, -0.9998, -1.0000],
        ...,
        [ 0.9956,  0.6980, -0.9738,  ..., -0.9182,  0.8446, -0.9987],
        [ 0.7540, -0.2676, -0.5729,  ...,  0.8997, -0.9931, -0.9276],
        [ 0.2238, -0.9767,  0.9896,  ..., -0.7383, -0.9845,  0.9978]])

In [16]:
h.shape

torch.Size([228146, 100])

In [17]:
w2 = torch.randn((100, 27))
b2 = torch.randn(27)

In [18]:
logits = h @ w2 + b2

In [19]:
logits.shape

torch.Size([228146, 27])

In [20]:
counts = logits.exp()

In [21]:
prob = counts / counts.sum(1, keepdims=True)

In [22]:
prob.shape

torch.Size([228146, 27])

In [23]:
loss = -prob[torch.arange(32), Y].log().mean()
loss

IndexError: shape mismatch: indexing tensors could not be broadcast together with shapes [32], [228146]

In [ ]:
# ------------ now made respectable :) ---------------

In [ ]:
Xtr.shape, Ytr.shape # dataset

g = torch.Generator().manual_seed(2147483647) # for reproducibility
C = torch.randn((27, 10), generator=g)
W1 = torch.randn((30, 200), generator=g)
b1 = torch.randn(200, generator=g)
W2 = torch.randn((200, 27), generator=g)
b2 = torch.randn(27, generator=g)
parameters = [C, W1, b1, W2, b2]

sum(p.nelement() for p in parameters) # number of parameters in total

for p in parameters:
    p.requires_grad = True

lri = []
lossi = []
stepi = []
loss_eva = []

In [24]:
#----------------------------------------
#new one
#----------------------------------------
Xtr.shape, Ytr.shape # dataset

g = torch.Generator().manual_seed(2147483647) # for reproducibility
C = torch.randn((27, 10), generator=g)
W1 = torch.randn((block_size*10, 200), generator=g)
b1 = torch.randn(200, generator=g)
W2 = torch.randn((200, 27), generator=g)
b2 = torch.randn(27, generator=g)
parameters = [C, W1, b1, W2, b2]

sum(p.nelement() for p in parameters) # number of parameters in total

for p in parameters:
    p.requires_grad = True

lri = []
lossi = []
stepi = []
loss_eva = []

In [28]:
words

['yuheng',
 'diondre',
 'xavien',
 'jori',
 'juanluis',
 'erandi',
 'phia',
 'samatha',
 'phoenix',
 'emmelynn',
 'hollan',
 'hollis',
 'callalily',
 'adelayde',
 'josephyne',
 'weldon',
 'kayle',
 'ragnar',
 'colbie',
 'taveon',
 'aki',
 'peyten',
 'kevari',
 'joella',
 'mecca',
 'egan',
 'justyce',
 'taliyah',
 'hayley',
 'alleah',
 'kymberlynn',
 'parrish',
 'houstyn',
 'jamaya',
 'ahmod',
 'nivin',
 'milli',
 'cristiana',
 'jaimee',
 'mitchell',
 'nairah',
 'lorena',
 'gentrie',
 'torrion',
 'savian',
 'benjamine',
 'airess',
 'knute',
 'sultana',
 'danai',
 'azzan',
 'issabelle',
 'abrahim',
 'aislyn',
 'aerys',
 'malaiya',
 'kieon',
 'ayansh',
 'berklee',
 'emmakate',
 'avaneesh',
 'amenadiel',
 'renardo',
 'brekken',
 'kamry',
 'wylin',
 'trixie',
 'breslynn',
 'bianka',
 'mordchai',
 'haoyu',
 'frankie',
 'josselin',
 'saila',
 'kionna',
 'jennings',
 'arlington',
 'jupiter',
 'aleisha',
 'kemorah',
 'denisse',
 'zada',
 'kaydynce',
 'bari',
 'darya',
 'ellieanne',
 'gretel',
 

In [ ]:
sum(p.nelement() for p in parameters)

In [ ]:
Xtr.shape

In [ ]:
Xtr[:5], Ytr[:5]

In [ ]:
C[0]

In [ ]:
Xtr[3].float() @ C[0]

In [ ]:
Xtr.shape, Ytr.shape

In [ ]:
lre = torch.linspace(-3, 0, 1000)
lrs = 10**lre

In [ ]:
from IPython.display import clear_output

In [31]:
Xtr.shape[0]

182625

In [ ]:
train_count = 300000
batch_size = 1024
steps_o_steps = 6
last_real_loss = 0
lr = 0.1

start = time.time()
for i in range(train_count):
  
  # minibatch construct
  ix = torch.randint(0, Xtr.shape[0], (batch_size,))
  
  # forward pass
  emb = C[Xtr[ix]] # (32, 3, 10)
  h = torch.tanh(emb.view(-1, block_size*10) @ W1 + b1) # (32, 200)
  logits = h @ W2 + b2 # (32, 27)
  loss = F.cross_entropy(logits, Ytr[ix])
  #print(loss.item())
  
  # backward pass
  for p in parameters:
    p.grad = None
  loss.backward()
  
  # update
  #lr = lrs[i]
  if (i%(10.0/steps_o_steps) == 0):
      lr = lr * 0.5
 
  #lr = 0.1 if i < 100000 else 0.01
  for p in parameters:
    p.data += -lr * p.grad

  # track stats
  #lri.append(lre[i])
  stepi.append(i)
  lossi.append(loss.log10().item())

  if i%(train_count*0.01) == 0:
    clear_output(wait=True)
    percentage = i/(train_count*0.1)
    print("<", '='*int(percentage), ' '*int(10-percentage) ,">", round((percentage*10), 1), "%")

  if i%(train_count*0.01) == 0:
    #track real loss
    emb = C[Xdev] # (32, 3, 2)
    h = torch.tanh(emb.view(-1, block_size*10) @ W1 + b1) # (32, 100)
    logits = h @ W2 + b2 # (32, 27)
    loss = F.cross_entropy(logits, Ydev)
    loss_eva.append(loss.log10().item())
    last_real_loss = loss.item()

runtime = time.time() - start
print("finished")
#print(loss.item())

In [ ]:
plt_count = 1

In [ ]:
plt.plot(stepi, lossi)
plt.plot(np.arange(0, train_count, train_count/100), loss_eva)
plt.title((f'a-{round(last_real_loss, 3)} c-{train_count} bs-{batch_size} rt-{round(runtime, 0)}s cl-{block_size} sos-{steps_o_steps}'))
plt.savefig(f'figs/{plt_count}.png')
plt_count = plt_count +1

In [ ]:
emb = C[Xtr] # (32, 3, 2)
h = torch.tanh(emb.view(-1, block_size*10) @ W1 + b1) # (32, 100)
logits = h @ W2 + b2 # (32, 27)
loss = F.cross_entropy(logits, Ytr)
loss.item()

In [ ]:
emb = C[Xdev] # (32, 3, 2)
h = torch.tanh(emb.view(-1, block_size*10) @ W1 + b1) # (32, 100)
logits = h @ W2 + b2 # (32, 27)
loss = F.cross_entropy(logits, Ydev)
loss.item()

In [ ]:
# visualize dimensions 0 and 1 of the embedding matrix C for all characters
plt.figure(figsize=(8,8))
plt.scatter(C[:,0].data, C[:,1].data, s=200)
for i in range(C.shape[0]):
    plt.text(C[i,0].item(), C[i,1].item(), itos[i], ha="center", va="center", color='white')
plt.grid('minor')

In [ ]:
# training split, dev/validation split, test split
# 80%, 10%, 10%

In [ ]:
# sample from the model
g = torch.Generator().manual_seed(2147483647 + 10)

for _ in range(200):
    
    out = []
    context = [0] * block_size # initialize with all ...
    while True:
      emb = C[torch.tensor([context])] # (1,block_size,d)
      h = torch.tanh(emb.view(1, -1) @ W1 + b1)
      logits = h @ W2 + b2
      probs = F.softmax(logits, dim=1)
      ix = torch.multinomial(probs, num_samples=1, generator=g).item()
      context = context[1:] + [ix]
      out.append(ix)
      if ix == 0:
        break
    
    print(''.join(itos[i] for i in out))